<a href="https://colab.research.google.com/github/syedmahmoodiagents/Agents/blob/main/RAG_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_community langchain_openai --q

In [11]:
!pip install -U langchain --q

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import os

In [ ]:
import os, getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass() # can be replace by Ollama settings

··········


In [ ]:
loader = PyPDFLoader("my_docs.pdf")       # change this to your document
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

In [ ]:
chunks = text_splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
retriever_model = SentenceTransformer('bert-base-nli-mean-tokens')
embeddings = retriever_model.encode(documents, convert_to_tensor=True)

In [ ]:
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:

faiss_store = FAISS.from_documents(chunks, embeddings)

# (Optional) Save FAISS index
faiss_store.save_local("faiss_index")


# faiss_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [ ]:
retriever = faiss_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

In [10]:

chat_history = []

# print("RAG Chatbot (FAISS) Ready! Ask questions about your document.\n")

while True:
    query = input("You: ")

    if query.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye!")
        break

    result = qa.invoke({"question": query, "chat_history": chat_history})

    print("\nBot:", result["answer"], "\n")

    # Save conversation history
    chat_history.append((query, result["answer"]))

    # Show retrieved chunks (optional)
    print("---- Retrieved Chunks ----")
    for doc in result["source_documents"]:
        print(doc.metadata, "\n", doc.page_content[:200], "...\n")
